In [4]:
import cv2
import numpy as np
import pdb

from Managers import *

# Make a Rect helper function that stores points

In [5]:
class PointsRectCallback:
    def __init__(self):
        self.p1 = None
        self.p2 = None

        
    def getRect(self):
        if not self.p1 or not self.p2: return

        x1, y1 = self.p1
        x2, y2 = self.p2
        x_min = min(x1, x2)
        x_max = max(x1, x2)
        y_min = min(y1, y2)
        y_max = max(y1, y2)
        return (x_min, y_min, x_max-x_min, y_max-y_min)
    
    
    def __call__(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if self.p1 is None:
                self.p1 = (x, y)
            elif self.p2 is None:
                self.p2 = (x, y)

# Mean Shift

Mean shift is like clustering but we find a kernel between pixels (one that includes colour properties in the colour distance as well). A group of pixels will move little between frames but the center of gravity will also move. We can't just track the object by taking every point and applying no discrimination, as it will pickup outlier pixels. 

We want to group together similiar pixels that are close and find a local peak. The peak indicates the cluster that the pixels belong to. The peak can change a bit over time as well, but we are modelling an object where pixels share some property and are reasonably close.

`masked_roi = cv2.inRange(self.hsv_roi, np.array([0., 0., 0.]), np.array([180., 255., 255.]))`

Masked histograms are used so that we know to ignore a certain thing in the image that we wish to ignore.

`self.roi_hist = cv2.calcHist([self.hsv_roi], [0], masked_roi, [180], [0,180])` <br>
`self.roi_hist = cv2.normalize(self.roi_hist, 0, 255, cv2.NORM_MINMAX)`

Open CV has a HSV range in 180 instead of 360 or 255 like some other libraries. In meanshift there is concept of distance that a cluster has moved. The histogram of the object needs to change by a relatively small amount between frames and the object must move a relatively small amount. The histogram is normalized between frames.

`self.term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)`

There is an iterative process between each frame to see how far the object as moved. The citertia is set out so that after a certain number of iterations the process stops and if the distance moved is 1 pixel or less.

`back_proj = cv2.calcBackProject([hsv], [0], self.roi_hist, [0, 180], 1)`

The back projection is a probability that a pixel belows to the current target histogram.

In [33]:
class HistogramTracker:
    def __init__(self, mousePoints, masked_lower=np.array([0., 0., 0.]), masked_upper=np.array([180., 255., 255.]), 
                 method=cv2.meanShift):
        self.mousePoints = mousePoints

        self.hsv_roi = None
        self.term_crit = None
        self.roi_hist = None
        self.track_window = None
        
        self.show_back_proj = False
                 
        self.masked_lower = masked_lower
        self.masked_upper = masked_upper
        self.method = method


    def intializeHSVRoi(self, image):
        x, y, w, h = self.track_window

        self.hsv_roi = image[y : y + h, x : x + w]
        self.hsv_roi = cv2.cvtColor(self.hsv_roi, cv2.COLOR_BGR2HSV)
        masked_roi = cv2.inRange(self.hsv_roi, self.masked_lower, self.masked_upper)

        self.roi_hist = cv2.calcHist([self.hsv_roi], [0], masked_roi, [180], [0,180])
        self.roi_hist = cv2.normalize(self.roi_hist, 0, 255, cv2.NORM_MINMAX)
        self.term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
 

    def trackHist(self, image):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        back_proj = cv2.calcBackProject([hsv], [0], self.roi_hist, [0, 180], 1)
    
        _, self.track_window = self.method(back_proj, self.track_window, self.term_crit)
        x, y, w, h = self.track_window

        if self.show_back_proj:
            image = back_proj
        cv2.rectangle(image, (x, y), (x + w, y + h), 255, 2)

        return image


    def __call__(self, image):
        if not self.track_window is None:
            image = self.trackHist(image)
            
            cv2.circle(image, self.mousePoints.p1, 2, (19, 134, 242), 5)
            cv2.circle(image, self.mousePoints.p2, 2, (19, 134, 242), 5)

        elif self.mousePoints.p2:
            self.track_window = self.mousePoints.getRect()
            self.intializeHSVRoi(image)

        elif self.mousePoints.p1:
            cv2.circle(image, self.mousePoints.p1, 2, (19, 134, 242), 5)

        return image

This technique can still fail. The histogram window does not change size with the object. If there is a strong colour such the background that dominates the original window, then the object can be lost. This is why masking is useful but the masking can make two objects look similiar that in a similiar brown colour range.

In [28]:
mousePoints = PointsRectCallback()
histTracker = HistogramTracker(mousePoints)

cam = CameraManager('Mean Shift', cv2.VideoCapture(0), histTracker, mouseCallback=mousePoints).addCloseCallback()
cam.writeVideo('D:/Meanshift.avi')
cam.run()

# CamShift

This the object can change size very quickly, and there are some colours that are close but do not look that similiar. If end up tracking a large object like a person that has a distinctive histogram.

In [36]:
mousePoints = PointsRectCallback()
histTracker = HistogramTracker(mousePoints, method=cv2.CamShift)

cam = CameraManager('Cam Shift', cv2.VideoCapture(0), histTracker, mouseCallback=mousePoints).addCloseCallback()
cam.writeVideo('D:/Camshift.avi')
cam.run()